In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler


In [9]:
def Segment(img):
    #using adaptiveThresholding technique
    thresh = cv2.adaptiveThreshold(img,1,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,39, -27)
    # Marker labelling
    ret, markers = cv2.connectedComponents(thresh)
    # First, find the contours:
    thresh = np.uint8(markers>0)
    contours,_ = cv2.findContours(thresh, 1, 2)
    cnt = [cnt for cnt in contours if cv2.contourArea(cnt)>10]
    return cnt, markers

def MajorAxis(contour):
    ellipse = cv2.fitEllipse(contour)
    (xc,yc),(d1,d2),angle = ellipse
    return max(d1,d2)/2
#Now calculate perimeter(L), Area(S), circularity = 4*pi*S/L^2, majorAxis length, MinorAxis length from the mask
def feature_extract(contours):
    features = []
    S_L = cv2.contourArea(contours[0])
    features.append(S_L)
    S_R = cv2.contourArea(contours[1])
    features.append(S_R)
    L_L = cv2.arcLength(contours[0],True)
    features.append(L_L)
    L_R = cv2.arcLength(contours[1],True)
    features.append(L_R)
    C_L = 4*np.pi*S_L/L_L**2
    features.append(C_L)
    C_R = 4*np.pi*S_R/L_R**2
    features.append(C_R)
    features.append(MajorAxis(contours[0]))
    features.append(MajorAxis(contours[1]))
    return features

def unpack(path, Labels):
    features =[]
    labels = []
    for label in Labels:
        cur_path = path + '/' + label
        cur_label = label
        #unpacking
        count=0
        for image in os.listdir(cur_path):
            if image.endswith(".png"):
                count+=1
                print(f"Processing Image-{image} in {label}")
                #read and convert to gray scale
                im = cv2.imread(cur_path+'/'+image)
                im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
                contours,markers = Segment(im)
                features.append(feature_extract(contours))
                labels.append(cur_label)
    return features, labels

def Accuracy(Prediction, labels):
    count=0
    for predicted, label in zip(Prediction, labels):
        if predicted == label:
            count+=1
    return 100*count/len(testing_labels)

In [10]:
training_path = 'Training'
training_labels = os.listdir(training_path)
training_features, training_labels = unpack(training_path, training_labels)



Processing Image-NC3008.png in NC
Processing Image-NC3011.png in NC
Processing Image-NC3013.png in NC
Processing Image-NC3016.png in NC
Processing Image-NC3053.png in NC
Processing Image-NC3055.png in NC
Processing Image-NC3071.png in NC
Processing Image-NC3074.png in NC
Processing Image-NC3075.png in NC
Processing Image-NC3085.png in NC
Processing Image-NC3087.png in NC
Processing Image-NC3100.png in NC
Processing Image-NC3103.png in NC
Processing Image-NC3104.png in NC
Processing Image-NC3109.png in NC
Processing Image-NC3114.png in NC
Processing Image-NC3151.png in NC
Processing Image-NC3156.png in NC
Processing Image-NC3157.png in NC
Processing Image-NC3160.png in NC
Processing Image-NC3161.png in NC
Processing Image-NC3171.png in NC
Processing Image-NC3172.png in NC
Processing Image-NC3188.png in NC
Processing Image-NC3191.png in NC
Processing Image-NC3201.png in NC
Processing Image-NC3202.png in NC
Processing Image-NC3204.png in NC
Processing Image-PD3012.png in PD
Processing Ima

In [11]:
testing_path = 'Testing'
testing_labels = os.listdir(training_path)
testing_features, testing_labels = unpack(testing_path, testing_labels)


Processing Image-NC3000.png in NC
Processing Image-NC3009.png in NC
Processing Image-NC3029.png in NC
Processing Image-NC3057.png in NC
Processing Image-NC3064.png in NC
Processing Image-NC3070.png in NC
Processing Image-NC3072.png in NC
Processing Image-NC3115.png in NC
Processing Image-NC3169.png in NC
Processing Image-NC3811.png in NC
Processing Image-NC3812.png in NC
Processing Image-NC3813.png in NC
Processing Image-NC3816.png in NC
Processing Image-PD3018.png in PD
Processing Image-PD3059.png in PD
Processing Image-PD3076.png in PD
Processing Image-PD3080.png in PD
Processing Image-PD3086.png in PD
Processing Image-PD3089.png in PD
Processing Image-PD3118.png in PD
Processing Image-PD3123.png in PD
Processing Image-PD3330.png in PD
Processing Image-PD3331.png in PD
Processing Image-PD3332.png in PD
Processing Image-PD3333.png in PD
Processing Image-PD3378.png in PD
Processing Image-PD3380.png in PD
Processing Image-PD3392.png in PD
Processing Image-PD3403.png in PD


In [12]:
print ("Training features: {}".format(np.array(training_features).shape))
print ("Training labels: {}".format(np.array(training_labels).shape))

Training features: (51, 8)
Training labels: (51,)


In [13]:
# Normalizing both data the data
scaler = StandardScaler()
scaler.fit(training_features)
training_data = scaler.transform(training_features)
testing_data = scaler.transform(testing_features)


In [14]:
# create the classifier
print("[STATUS] Creating the classifier..")
clf_svm = LinearSVC(random_state=9, max_iter=1000)
# fit the training data and labels
print("[STATUS] Fitting data/label to model..")
clf_svm.fit(training_data, training_labels)

[STATUS] Creating the classifier..
[STATUS] Fitting data/label to model..


LinearSVC(random_state=9)

In [15]:
#prediction
Prediction = [clf_svm.predict(feature.reshape(1, -1))[0] for feature in testing_data]
model_accuracy = Accuracy(Prediction, testing_labels)
print(model_accuracy)

96.55172413793103


In [22]:
for p,l in zip(Prediction, testing_labels):
    print(f'label {l} predicted as {p}')

label NC predicted as NC
label NC predicted as PD
label NC predicted as NC
label NC predicted as NC
label NC predicted as NC
label NC predicted as NC
label NC predicted as NC
label NC predicted as NC
label NC predicted as NC
label NC predicted as NC
label NC predicted as NC
label NC predicted as NC
label NC predicted as NC
label PD predicted as PD
label PD predicted as PD
label PD predicted as PD
label PD predicted as PD
label PD predicted as NC
label PD predicted as PD
label PD predicted as PD
label PD predicted as PD
label PD predicted as PD
label PD predicted as PD
label PD predicted as PD
label PD predicted as PD
label PD predicted as PD
label PD predicted as PD
label PD predicted as PD
label PD predicted as PD
